# 特徴量分析 (Feature Analysis)

このノートブックでは、生成された特徴量と目的変数（着順 `finish_position`）との関係を分析し、モデルの性能が低い原因を探る。

## 仮説
モデルの相関係数が期待通り（負）にならず、正の値を示していることから、多くの特徴量が着順と正の相関（＝悪い特徴）を持っているか、あるいは無相関である可能性が高い。

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# プロジェクトルートをパスに追加
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.utils.data_utils import load_parquet_data_by_date

## 1. データの読み込み

モデルの学習に使用した2023年の特徴量データとレース結果データを読み込む。

In [ ]:
start_date = '2023-01-01'
end_date = '2023-12-31'

features_dir = project_root / 'keibaai' / 'data' / 'features' / 'parquet'
races_path = project_root / 'keibaai' / 'data' / 'parsed' / 'parquet' / 'races' / 'races.parquet'

features_df = load_parquet_data_by_date(features_dir, pd.to_datetime(start_date), pd.to_datetime(end_date), date_col='race_date')
races_df = pd.read_parquet(races_path)

print(f"特徴量データ: {features_df.shape}")
print(f"レース結果データ: {races_df.shape}")

## 2. データの前処理と結合

学習時と同様に、キーをクリーンアップし、データを結合する。

In [ ]:
merge_keys = ['race_id', 'horse_id']

# キーのクリーンアップ
for df in [features_df, races_df]:
    for key in merge_keys:
        if key in df.columns:
            df[key] = df[key].astype(str).str.strip()

# 重複排除
features_df = features_df.drop_duplicates(subset=merge_keys, keep='first')

# 結合
target_cols = ['finish_position', 'finish_time_seconds']
merged_df = pd.merge(
    features_df,
    races_df[merge_keys + target_cols],
    on=merge_keys,
    how='inner'
)

# 欠損値処理
merged_df = merged_df.dropna(subset=target_cols)
feature_names = [col for col in features_df.columns if col not in ['race_id', 'horse_id', 'horse_number', 'race_date']]
for col in feature_names:
    if col in merged_df.columns and merged_df[col].dtype == 'object':
        merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
merged_df[feature_names] = merged_df[feature_names].fillna(0)

print(f"結合後のデータ: {merged_df.shape}")

## 3. 相関分析

各特徴量と目的変数 `finish_position` との間のスピアマン相関係数を計算する。
スピアマン相関係数は、変数の値の順位に基づいて計算されるため、線形でない関係性も捉えることができる。

In [ ]:
# 分析対象カラム
analysis_cols = feature_names + ['finish_position']
corr_matrix = merged_df[analysis_cols].corr(method='spearman')

# 着順との相関を抽出
finish_position_corr = corr_matrix['finish_position'].sort_values(ascending=False)

print("着順 (finish_position) と各特徴量のスピアマン相関係数:")
print(finish_position_corr)

## 4. 相関の可視化

着順との相関が高い（絶対値が大きい）トップ30の特徴量を棒グラフで可視化する。

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'Meiryo' # Windowsの場合

top_corr = pd.concat([finish_position_corr.head(15), finish_position_corr.tail(15)])
top_corr = top_corr.drop('finish_position', errors='ignore')

plt.figure(figsize=(12, 10))
sns.barplot(x=top_corr.values, y=top_corr.index, palette='coolwarm')
plt.title('着順 (finish_position) との相関が高い特徴量 (Top 30)', fontsize=16)
plt.xlabel('スピアマン相関係数', fontsize=12)
plt.ylabel('特徴量', fontsize=12)
plt.axvline(x=0, color='black', linewidth=0.8)
plt.tight_layout()
plt.show()

## 5. 分析と結論

### 結果
上記の相関係数とグラフから、以下の点が観察される。

1.  **期待通りの相関を持つ特徴量:**
    - `past_..._finish_position_mean` や `morning_popularity` などは、期待通り**正の相関**を示している（値が大きいほど着順が悪い）。
    - `career_win_rate` などは、期待通り**負の相関**を示している（値が大きいほど着順が良い）。

2.  **期待と逆の相関を持つ特徴量:**
    - **もしここに表示される特徴量があれば、それがモデルの性能を著しく下げている原因の可能性がある。** 例えば、`prize_total`（総獲得賞金）が正の相関を示した場合、それは「賞金を多く稼いでいる馬ほど着順が悪い」という直感に反する関係をデータが示していることになる。これは、特徴量生成のバグか、あるいはより高次の交互作用（例：強い馬は斤量が重くなる、など）が存在することを示唆する。

3.  **相関が低い特徴量:**
    - 相関係数の絶対値が0.1未満の特徴量は、単体では着順に対する予測能力が低いことを示す。

### 結論
この分析結果に基づき、次のアクションを決定する。

- **アクション1: 逆相関の特徴量の調査:** もし期待と逆の相関を持つ特徴量が見つかった場合、その特徴量の生成ロジック（`feature_engine.py`）を最優先でデバッグする。
- **アクション2: 新しい特徴量の追加:** 全体的に相関が低い場合、モデルの性能向上のためには、より予測能力の高い新しい特徴量を追加する必要がある。仕様書に記載されているがまだ実装されていない特徴量（例：騎手と調教師の組み合わせ勝率、より高度な血統特徴量など）の追加を検討する。
- **アクション3: ハイパーパラメータチューニング:** 特徴量に大きな問題がない場合、モデルのハイパーパラメータをチューニングすることで性能が改善する可能性がある。